In [49]:
import tensorflow as tf
print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.11.0


* Tensorflow 2.0 시작하기 : Beginners

1. 사전에 빌드한 dataset을 로드한다.
2. 이미지를 분류하는 Neural Network ML model을 빌드한다.
3. model을 Training
4. model의 accuracy를 평가한다.

* MNIST Dataset을 로드하고 준비한다. Sample Data를 integer에서 floating point number로 변환한다.

In [50]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255, x_test/255

* ML Model 빌드하기
- layer을 차례대로 쌓아 tf.keras.Sequential model을 만든다. 훈련에 사용할 optimizer와 loss function을 선택한다.

In [52]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [53]:
# 각 예시에서 model은 각 class에 대해 하나씩, logits vector를 반환한다.
predictions = model(x_train[:1])
predictions

2023-11-11 01:51:40.748366: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[0.14846037, 0.07245988, 0.09627186, 0.07427168, 0.04530562,
        0.1361638 , 0.13757524, 0.14994   , 0.05409979, 0.08545171]],
      dtype=float32)>

In [54]:
predictions.numpy

<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[0.14846037, 0.07245988, 0.09627186, 0.07427168, 0.04530562,
        0.1361638 , 0.13757524, 0.14994   , 0.05409979, 0.08545171]],
      dtype=float32)>>

In [55]:
# tf.nn.softmax 함수는 다음과 같이 이러한 logits을 각 class에 대한 확류로 변환한다.
tf.nn.softmax(predictions).numpy

<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[0.10489027, 0.09721395, 0.09955659, 0.09739024, 0.09460969,
        0.10360837, 0.10375471, 0.10504558, 0.09544538, 0.09848518]],
      dtype=float32)>>

* 참고
- tf.nn.softmax 함수를 network의 마지막 layer에 대한 activation function으로 베이킹할 수 있다. 이렇게 하면 model output을 더 직접적으로 해석할 수 있지만 이 접근법은 softmax output을 사용할 경우 모든 model에 대해 정확하고 수치적으로 안정적인 loss 계산을 제송하는 것이 불가능하므로 권장하지 않는다.

In [56]:
# tf.keras.losses.SparseCategoricalCrossentropy 함수는 logits의 vector와 True index를 사용하고 각 예시에 대해 스칼라 loss를 반환하는 훈련용 손실 함수 이다.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

- 이 loss는 실제 클래스의 음의 로그 확률과 같다. model이 올바른 클래스를 확신하는 경우 loss는 0이다
- 이 훈련되지 않은 모델은 무작위에 가까운 확률(각 클래스에 대해 1/10)을 제공하므로 초기 손실은 tf.math.log(1/10) ~= 2.3에 근접해야 한다.

In [61]:
loss_fn(y_train[:1], predictions).numpy()

1.9938967

* 훈련을 시작하기 전에 Keras Model.compile을 사용하여 model을 구성하고 compile한다.

- optimizer 클래스를 adam으로 설정하고 loss를 앞에서 정의한 loss_fn 함수로 설정한다. metrics 매개변수를 accuracy로 설정하여 model에 대해 평가할 metric을 지정한다.

In [62]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

* Model 훈련 및 평가하기

In [63]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5


/home/jisoo/.local/lib/python3.10/site-packages/keras/backend.py:5585: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2023-11-11 02:01:56.596118: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fd258031bb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-11 02:01:56.596190: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2023-11-11 02:01:56.633441: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-11 02:01:56.987067: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for 

1875/1875 [==============================] - 11s 5ms/step - loss: 0.2927 - accuracy: 0.9160
Epoch 2/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1396 - accuracy: 0.9578
Epoch 3/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1079 - accuracy: 0.9679
Epoch 4/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0870 - accuracy: 0.9730
Epoch 5/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0747 - accuracy: 0.9768


In [65]:
model.evaluate(x_test, y_test, verbose=2)

/home/jisoo/.local/lib/python3.10/site-packages/keras/backend.py:5585: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


313/313 - 1s - loss: 0.0743 - accuracy: 0.9772 - 1s/epoch - 4ms/step


[0.07434934377670288, 0.9771999716758728]

* 훈련된 이미지 분류기는 이 Dataset에서 약 97%의 accuracy를 달성한다.

* Model이 확률을 반환하도록 하려면 다음과 같이 훈련된 model을 래핑하고 여기에 softmax를 첨부할 수 있다.

In [66]:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

In [81]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0.08533683, 0.08533683, 0.08533684, 0.08533739, 0.08533683,
        0.08533683, 0.08533683, 0.23196799, 0.08533683, 0.08533686],
       [0.08533967, 0.08535551, 0.23192306, 0.0853425 , 0.08533967,
        0.08533968, 0.08533967, 0.08533967, 0.08534094, 0.08533967],
       [0.08533998, 0.23191819, 0.08534138, 0.08534021, 0.08534122,
        0.08534012, 0.08534101, 0.08534957, 0.08534832, 0.08533999],
       [0.23195562, 0.08533762, 0.08533923, 0.08533762, 0.08533768,
        0.08533763, 0.08534151, 0.08533764, 0.08533762, 0.08533789],
       [0.08535497, 0.08535123, 0.08535279, 0.08535124, 0.23174   ,
        0.08535171, 0.08535279, 0.08541127, 0.08535132, 0.08538269]],
      dtype=float32)>